[![Drawdown](data/images/drawdown_logo_clouds2.png)](https://drawdown.org)

**[Project Drawdown](https://drawdown.org)** is a living research and communications organization that assesses, maps, models, and describes the potential of the most substantive solutions to achieve drawdown by 2050. It is a collaborative effort of over 200 researchers, policymakers, businesses, thought leaders, and organizations developing a model to enable action and implementation throughout the world. Our common mission is to do our part in solving global warming, by enabling a new, regenerative ‘business-as-usual’ that has cascading benefits to human well-being in concert with nature, while training the next generation of global citizens and thought-leaders. 

Our aim is to democratize and extend this model and approach to collective impact far and wide. To achieve that, we have been working with partners across the world to help seed replications and the expansion of the model in-place, to nurture and support a thriving ecosystem based on self-organizing, networked sparks wherever Drawdown has taken root.

**To use the model engine** run the first code cell to generate an overview of available solutions, each with a checkbox to generate detailed results. After selecting the desired solution(s), run the code cell below the overview to generate the models and see the detailed results.

In [ ]:
import ui.charts
j = ui.charts.JupyterUI()
j.get_overview()

In [ ]:
display(j.get_detailed_results_tabs())